In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install scanpy anndata

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfTransformer
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import numpy as np
import scanpy as sc
import warnings
warnings.filterwarnings("ignore")

In [4]:
def preview(adata):
    print("\nUns:", adata.uns)
    print("Observations:", adata.obs.head())
    print("\nVariables:", adata.var.head())
    print("Shape:", adata.shape)

In [5]:
def load_and_subsample(file_path, n_obs):
    adata = sc.read(file_path)
    sc.pp.subsample(adata, n_obs=n_obs, random_state=42)
    return adata

def generate_sample_dataset():
    base_path = '/content/drive/MyDrive/modality prediction/GEX2ATAC/'
    datasets = ['train_mod1', 'train_mod2']
    n_obs = 5000

    for dataset in datasets:
        file_path = base_path + f'{dataset}.h5ad'
        sampled_file_path = f'/content/drive/MyDrive/modality prediction/GEX2ATAC/sampled_{dataset}.h5ad'

        loaded_data = load_and_subsample(file_path, n_obs)
        loaded_data.write(sampled_file_path)


In [6]:
def dense_data_generation(adata_mod1_train_sample, adata_mod2_train_sample, adata_mod1_test_sample, adata_mod2_test_sample):
  # extract data matrices for training and testing
  train_X = adata_mod1_train_sample.X
  train_y = adata_mod2_train_sample.X
  test_X = adata_mod1_test_sample.X
  test_y = adata_mod2_test_sample.X

  # convert to dense
  train_X = train_X.toarray()
  train_y = train_y.toarray()
  test_X = test_X.toarray()
  test_y = test_y.toarray()
  return train_X, train_y, test_X, test_y

In [7]:
def RMSE(test_y, predictions):
  # evaluate the model on the original testing data
  rmse = mean_squared_error(test_y, predictions, squared=False)
  print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')

In [8]:
def linear_regression(train_X, train_y, test_X, test_y):
  # train a linear regression model
  model = LinearRegression()
  model.fit(train_X, train_y)
  predictions = model.predict(test_X)

  # evaluate the model
  RMSE(test_y, predictions)

In [9]:
def SVD_linear_regression(train_X, train_y, test_X, test_y):
  # perform SVD on the training data
  num_components = 50
  svd_X = TruncatedSVD(n_components=num_components)
  svd_y = TruncatedSVD(n_components=num_components)
  lsi_matrix_train_X = svd_X.fit_transform(train_X)
  lsi_matrix_train_y = svd_y.fit_transform(train_y)

  # train a linear regression model
  model = LinearRegression()
  model.fit(lsi_matrix_train_X, lsi_matrix_train_y)

  # transform the testing data using the same SVD transformation
  lsi_matrix_test_X = svd_X.transform(test_X)

  # make predictions on the testing data
  lsi_predictions_y = model.predict(lsi_matrix_test_X)
  predictions = svd_y.inverse_transform(lsi_predictions_y)

  # evaluate the model
  RMSE(test_y, predictions)

In [10]:
def TFIDF_linear_regression(train_X, train_y, test_X, test_y):
  # apply TF-IDF to training and testing data
  tfidf_transformer_X = TfidfTransformer()
  tfidf_transformer_y = TfidfTransformer()
  tfidf_train_X = tfidf_transformer_X.fit_transform(train_X)
  tfidf_train_y = tfidf_transformer_y.fit_transform(train_y)


  # perform SVD on the training data
  num_components = 50
  svd_X = TruncatedSVD(n_components=num_components)
  svd_y = TruncatedSVD(n_components=num_components)
  lsi_matrix_train_X = svd_X.fit_transform(tfidf_train_X)
  lsi_matrix_train_y = svd_y.fit_transform(tfidf_train_y)


  # train a linear regression model
  model = LinearRegression()
  model.fit(lsi_matrix_train_X, lsi_matrix_train_y)

  # transform the testing data using the same TFIDF vectorizer and SVD transformation
  tfidf_test_X = tfidf_transformer_X.transform(test_X)
  lsi_matrix_test_X = svd_X.transform(tfidf_test_X)
  predictions = model.predict(lsi_matrix_test_X)

  # inverse transform
  inverse_svd_predictions = svd_y.inverse_transform(predictions)
  inverse_tfidf_predictions = inverse_svd_predictions * (1 / tfidf_transformer_y.idf_)

  # evaluate the model
  RMSE(test_y, inverse_tfidf_predictions)

In [11]:
# Define autoencoder model
class Autoencoder(nn.Module):
  def __init__(self, input_dim, encoding_dim):
    super(Autoencoder, self).__init__()
    self.encoder = nn.Sequential(
      nn.Linear(input_dim, encoding_dim),
      nn.ReLU()
      )
    self.decoder = nn.Sequential(
      nn.Linear(encoding_dim, input_dim),
      nn.Sigmoid()
      )

  def forward(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

def autoencoder_linear_regression(train_X, train_y, test_X, test_y):
  # train the autoencoder
  input_dim = train_X.shape[1]
  encoding_dim = 50
  autoencoder = Autoencoder(input_dim, encoding_dim)
  criterion = nn.MSELoss()
  optimizer = optim.Adam(autoencoder.parameters(), lr=0.01)
  num_epochs = 10

  for epoch in range(num_epochs):
    inputs = torch.FloatTensor(train_X)
    optimizer.zero_grad()
    outputs = autoencoder(inputs)
    loss = criterion(outputs, inputs)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

  # encode the training and testing data using the autoencoder
  autoencoder.eval()
  encoded_train_X = autoencoder.encoder(torch.FloatTensor(train_X)).detach().numpy()
  encoded_test_X = autoencoder.encoder(torch.FloatTensor(test_X)).detach().numpy()

  # train a linear regression model
  linear_regression(encoded_train_X, train_y, encoded_test_X, test_y)


def autoencoder_SVD_linear_regression(train_X, train_y, test_X, test_y):
  # get SVD
  num_components = 50
  svd_X = TruncatedSVD(n_components=num_components)
  svd_y = TruncatedSVD(n_components=num_components)
  lsi_matrix_train_X = svd_X.fit_transform(train_X)
  lsi_matrix_train_y = svd_y.fit_transform(train_y)
  lsi_matrix_test_X = svd_y.transform(test_y)

  # train the autoencoder
  input_dim = lsi_matrix_train_X.shape[1]
  encoding_dim = 50
  autoencoder = Autoencoder(input_dim, encoding_dim)
  criterion = nn.MSELoss()
  optimizer = optim.Adam(autoencoder.parameters(), lr=0.01)
  num_epochs = 10

  for epoch in range(num_epochs):
    inputs = torch.FloatTensor(lsi_matrix_train_X)
    optimizer.zero_grad()
    outputs = autoencoder(inputs)
    loss = criterion(outputs, inputs)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

  # encode the training and testing data using the autoencoder
  autoencoder.eval()
  encoded_train_X = autoencoder.encoder(torch.FloatTensor(lsi_matrix_train_X)).detach().numpy()
  encoded_test_X = autoencoder.encoder(torch.FloatTensor(lsi_matrix_test_X)).detach().numpy()

  # train a linear regression model
  model = LinearRegression()
  model.fit(encoded_train_X, lsi_matrix_train_y)

  # make predictions on the testing data
  lsi_predictions_y = model.predict(encoded_test_X)
  predictions = svd_y.inverse_transform(lsi_predictions_y)

  # evaluate the model
  RMSE(test_y, predictions)

In [12]:
def main():
  # load the original training file
  file_path = '/content/drive/MyDrive/modality prediction/GEX2ATAC/train_mod1.h5ad'
  adata = sc.read(file_path)

  # preview basic information
  preview(adata)

  # generate sample dataset
  generate_sample_dataset()

  # read the sample dataset
  mod1_train_sample = sc.read('/content/drive/MyDrive/modality prediction/GEX2ATAC/sampled_train_mod1.h5ad')
  mod2_train_sample = sc.read('/content/drive/MyDrive/modality prediction/GEX2ATAC/sampled_train_mod2.h5ad')
  mod1_test_sample = sc.read('/content/drive/MyDrive/modality prediction/GEX2ATAC/test_mod1.h5ad')
  mod2_test_sample = sc.read('/content/drive/MyDrive/modality prediction/GEX2ATAC/test_mod2.h5ad')
  print('mod1_train shape:', mod1_train_sample.shape)
  print('mod2_train shape:', mod2_train_sample.shape)

  # convert sparse matrices to df for better handling
  train_X, train_y, test_X, test_y = dense_data_generation(mod1_train_sample, mod2_train_sample, mod1_test_sample, mod2_test_sample)
  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

  # GEX2ATAC
  print("For GEX to ATAC using linear regression only")
  linear_regression(train_X, train_y, test_X, test_y)
  print("For GEX to ATAC using SVD")
  SVD_linear_regression(train_X, train_y, test_X, test_y)
  print("For GEX to ATAC using TFIDF")
  TFIDF_linear_regression(train_X, train_y, test_X, test_y)
  print("For GEX to ATAC using autoencoder")
  autoencoder_linear_regression(train_X, train_y, test_X, test_y)
  print("For GEX to ATAC using SVD and autoencoder")
  autoencoder_SVD_linear_regression(train_X, train_y, test_X, test_y)

  # ATAC2GEX
  print("For ATAC to GEX using linear regression only")
  linear_regression(train_y, train_X, test_y, test_X)
  print("For ATAC to GEX using SVD")
  SVD_linear_regression(train_y, train_X, test_y, test_X)
  print("For ATAC to GEX using TFIDF")
  TFIDF_linear_regression(train_y, train_X, test_y, test_X)
  print("For ATAC to GEX using autoencoder")
  autoencoder_linear_regression(train_y, train_X, test_y, test_X)
  print("For ATAC to GEX using SVD and autoencoder")
  autoencoder_SVD_linear_regression(train_y, train_X, test_y, test_X)

In [13]:
if __name__ == "__main__":
    main()


Uns: {'dataset_id': 'openproblems_bmmc_multiome_phase1v2_PM_gex2atac', 'organism': 'human'}
Observations:                         batch  size_factors
TAGTTGTCACCCTCAC-1-s1d1  s1d1      0.453484
CTATGGCCATAACGGG-1-s1d1  s1d1      0.455631
CCGCACACAGGTTAAA-1-s1d1  s1d1      2.435348
TCATTTGGTAATGGAA-1-s1d1  s1d1      0.347226
ACCACATAGGTGTCCA-1-s1d1  s1d1      0.534205

Variables:                    gene_ids feature_types
AL627309.5  ENSG00000241860           GEX
LINC01409   ENSG00000237491           GEX
LINC01128   ENSG00000228794           GEX
NOC2L       ENSG00000188976           GEX
KLHL17      ENSG00000187961           GEX
Shape: (30329, 13431)
mod1_train shape: (5000, 13431)
mod2_train shape: (5000, 10000)
For GEX to ATAC using linear regression only
Root Mean Squared Error (RMSE): 0.1909
For GEX to ATAC using SVD
Root Mean Squared Error (RMSE): 0.1385
For GEX to ATAC using TFIDF
Root Mean Squared Error (RMSE): 0.1443
For GEX to ATAC using autoencoder
Epoch 1/10, Loss: 0.287326663